In [39]:
import numpy as np
from scipy.special import comb
from itertools import combinations
from scipy.io import mmread, mminfo
from os import listdir
from os.path import isfile, join

### Start of Elimination Ordering.

#### Normalize Stage

In [40]:
'''Normalize Stage'''
'''preliminaries:
- node == vertex ("vertices" for plural)
- nodes' index start at 0
- all graphs and trees are represented in matrix farm
- valency is the sum of the edges' weights of a node, i.e. the sum of the row in a corresponding index, in numpy:
    np.sum(graph[node_idx])
- currently, the matrix is assumed to be symmetric (undirected)
- a fill is only calculated during the elimination, not during the algorithm process to get the elimination ordering,
and a fill is A[i][j] = 1 iff A[i][k] = A[k][i] = A[j][k] = A[k][j] = 1 then A[i][k] = A[k][i] = A[j][k] = A[k][j] = 0
- the diagonal element is always ignored (must be zero'd first if it's nonzero)
- non zeros' values doesn't matter (unweighted), it's always binary (0,1)
'''
'''independent functions:'''

#for transforiming tree matrices to ordered list
def topological_sort_tree(tree_in):
    #look for the "first" node, which is the node with no incoming edges:
    tree = np.copy(tree_in) #copy the tree so the input wont be affected
    size = tree.shape[0]
    S = []
    for i in range(size):
        #need to exclude disconnected nodes by checking row-wise too:
        if np.sum(tree[:,i]) == 0:
            if np.sum(tree[i]) > 0:
                S.append(i)
    print("S",S)
    
    enque = lambda q, x: q.append(x)
    deque = lambda q: q.pop(0)
    #kahn's algorithm for topological sort (https://en.wikipedia.org/wiki/Topological_sorting):
    #input: tree, first_nodes
    L = []
    while len(S) > 0:
        n = deque(S)
        L.append(n)
        ms = np.where(tree[n] == 1)[0]    #look for set of destination nodesf rom n (neighbours)
        #for each node m with an edge e from n to m:
        for m in ms:
            tree[n][m] = 0 #remove edge e from the graph
            if np.sum(tree[:,m]) == 0: #if m has no other incoming edges then
                enque(S, m) #insert m into S
    #there should be a final check whether the graph still has some edges, but it isnt necessary for tree cases since trees wont have DAG
    return L

#Breadth-First-Search traversal algorithm:
def BFS(graph, source):
    n_init = graph.shape[0]
    q = []
    enque = lambda q, x: q.append(x)
    deque = lambda q: q.pop(0)
    visited = np.array([0]*n_init)
    distances = np.array([0]*n_init)
    visited[source] = 1
    enque(q, source)
    q_counter = 1 #to keep track how many neighbours enqueued
    path = []
    while q:
        v = deque(q)
        q_counter = q_counter - 1
        neighbours = np.where(graph[v] == 1)[0] #enque all v's neighbours (gamma(v))
        for node_i in neighbours:
            if (visited[node_i] == 0) and (node_i not in q):
                enque(q, node_i)
                visited[node_i] = 1
                q_counter += 1
        #print(v, neighbours, q, q_counter, depth)
        path.append(v)
    return path

#get ordered list from merge forest
def get_ordered_list_merged_vertex(tree, placed_vertex):
    '''algorithm for tree-tracing that covers all scenarios:
    0. transpose the tree (to get the reverse order), due to the nature of the merge procedure, the leaves will be the roots
    1. topological sort to get the root(s)
    2. determine the roots by checking the connections between vertices
    3. if there are more than one roots:
        BFS traverse starting from the placed node to get the ordered lists
    else:
        just use the list from the topological sort as the ordered list
    '''
    topological_list = topological_sort_tree(tree.T)
    '''
    print("topological_list",topological_list)
    print(tree)
    print(tree.T)
    '''
    #check the number of roots and get the corresponding roots:
    length = len(topological_list)
    roots = [topological_list[0]]
    for i_elem in topological_list:
        non_root_found = False
        for j_elem in topological_list:
            if i_elem != j_elem:
                if tree.T[i_elem][j_elem] == 0:
                    #print(i_elem, j_elem)
                    roots.append(j_elem)
                else:
                    non_root_found = True
                    break
        if non_root_found:
            break
    #print("roots",roots)
    #if more than one roots, do BFS starting from the placed node, else just use the topological_list:
    ordered_list = None
    if len(roots) > 1:
        ordered_list = BFS(tree.T, placed_vertex)
        #print("orderedlist",ordered_list)
    else:
        ordered_list = topological_list
        #print("orderedlist",topological_list)
    return ordered_list

#clique checker:
def clique_check(graph, vertices_idx):
    #get subgraph, by slicing indexes:
    subgraph = graph[vertices_idx][:,vertices_idx]
    n = subgraph.shape[0]
    #check for clique from subgraph:
    upper_tri = subgraph[np.triu_indices(n, 1)]
    return np.sum(upper_tri) == comb(n, 2)

#subset checker:
def check_subset(graph, neighbours):
    bool_subset = False
    j_get = None
    for j_node in neighbours:
        #probably need to be stopped earlier? instead of taking the last neighbour index
        gamma_j = np.where(graph[j_node] == 1)[0]
        j_T = np.append(gamma_j, j_node) #j^up_tack = j union gamma(j):= j added to its neighbours
        if set(neighbours).issubset(set(j_T)): #gamma(i) \subset j^up_tack
            bool_subset = True
            j_get = j_node
            break #stop when found
    return bool_subset, j_get

#more accurate way of checking the total nodes within a graph, since the edge is represented 
#by the value of A[i][j] cell, e.g if i <-> j is connected, it means A[i][j] = A[j][i] 1, otherwise 0, 
#so the size of the matrix may not correspond to the total number of nodes
def get_total_nodes(graph, row_size):
    counter = 0
    for i in range(row_size):
        if np.sum(graph[i]) > 0:
            counter += 1
    return counter

'''end'''


def initialize(graph):
    '''function for data initialization, returns:
    - e vector placeholder
    - weight vector w
    - empty merge forest
    - first zero and last zero idxes
    - deleted bool array
    '''
    n = graph.shape[0]
    e = np.array([-1]*n) #for now the placeholder is an array of -1
    w = np.array([1]*n) #weight vector for merge forest
    merge_forest = np.zeros((n,n)) #merge forest for assessment criteria
    deleted = np.array([False]*n)
    first_zero = 0; last_zero = -1
    return e, w, first_zero, last_zero, merge_forest, deleted

#normalize stage:
def normalize(graph):
    global deleted, e, w, first_zero, last_zero, merge_forest
    n = n_init = graph.shape[0] #number of nodes
    '''e = np.array([-1]*n) #for now the placeholder is an array of -1
    w = np.array([1]*n) #weight vector for merge forest
    merge_forest = np.zeros((n,n)) #merge forest for assessment criteria'''
    modified = np.array([1]*n) #modified = 1, otherwise 0'''


    #normalize stage
    #for now, cyclic ordering assumption: start from the 1st index to last idx, hence for-loop
    #need merge check for every node passed, by w[i] > 1
    print("i, n, m, valency, valencies, e, modified, firstzero, lastzero")
    while np.sum(modified) > 0:
        print()
        #for i in range(n_init):
        for i in range(n_init):
            #check if it is already deleted, if yes, skip:
            if deleted[i]: #deleted in prev round
                modified[i] = 0 #set modified to 0
                print("already deleted:",i)
                continue
            if np.sum(modified) == 0:
                break
            #recalculate all of the values:
            n = get_total_nodes(graph, n_init) #recalculate n by excluding zero vectored rows (disconnected vertices)
            valencies = np.array([np.sum(graph[j]) for j in range(n_init)]) #needs to recalculate the valency for each update due to the graph-change
            #print(i,n,m,valency,valencies,e,modified)
            mean_valency = np.sum(valencies)/n #get mean valency
            max_valency = np.max(valencies) #get max valency
            valency = np.sum(graph[i]) #get vertex's valency
            m = np.min([mean_valency, np.floor(n**(1/4) + 3)])
            #m = np.floor(n**(1/4) + 3) #probably this is the correct interpretiation
            print("mean_valency, np.floor(n**(1/4) + 3)",mean_valency, np.floor(n**(1/4) + 3))
            neighbours = np.where(graph[i] == 1)[0] #get the neighbours of i
            print(i,n,m,valency,valencies,e,modified,np.sum(modified),first_zero, last_zero)
            print("neighbours",neighbours)
            #check all of the conditions based on the valency
            if valency == n-1:
                ##always check for merge - i.e w[i] > 1
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[len_e + last_zero - len_ord_list + 1 : len_e + last_zero + 1] = ordered_list #lastzero placement
                    last_zero -= len_ord_list #decrement last zero by the size of the ordered list 
                else:
                    #add to the last zero and decrement the indexer:
                    e[last_zero] = i
                    last_zero -= 1
                graph[i] = graph[:,i] = 0  #remove from graph by deleting edges
                deleted[i] = True
                #graph = np.delete(graph, i, 0) #delete from graph -- this should be the proper deletion method, though not sure if it's faster
                #graph = np.delete(graph, i, 1)
                modified[neighbours] = 1 #set neighbours as modified
                print("rule 1")
            elif (valency > np.ceil(n/2)) and (valency == max_valency):
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[len_e + last_zero - len_ord_list + 1 : len_e + last_zero + 1] = ordered_list
                    last_zero -= len_ord_list
                else:
                    e[last_zero] = i
                    last_zero -= 1
                graph[i] = graph[:,i] = 0
                deleted[i] = True
                modified[neighbours] = 1
                print("rule 2")
            elif valency <= 1:
                #e.insert(0, i) #place vertex first
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[first_zero : first_zero + len_ord_list] = ordered_list #insert by firstzero pos
                    first_zero += len_ord_list #increment the first zero by the size of the ordered list
                else:
                    #add to the first zero pos and increment the indexer:
                    e[first_zero] = i
                    first_zero += 1
                graph[i] = graph[:,i] = 0
                deleted[i] = True
                modified[neighbours] = 1
                print("rule 3")
            elif valency == 2:
                #e.insert(0, i)
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[first_zero : first_zero + len_ord_list] = ordered_list #insert by firstzero pos
                    first_zero += len_ord_list
                else:
                    #add to the first zero pos and increment the indexer:
                    e[first_zero] = i
                    first_zero += 1
                graph[neighbours[0]][neighbours[1]] = graph[neighbours[1]][neighbours[0]] = 1 #make edge between them -- fill the value of the cell with 1
                graph[i] = graph[:,i] = 0
                deleted[i] = True
                modified[neighbours] = 1
                print("rule 4")
            elif (valency <= m) and (clique_check(graph, neighbours)):
                #e.insert(0, i)
                if w[i] > 1:
                    ordered_list = get_ordered_list_merged_vertex(merge_forest, i)
                    len_e = len(e)
                    len_ord_list = len(ordered_list)
                    e[first_zero : first_zero + len_ord_list] = ordered_list #insert by firstzero pos
                    first_zero += len_ord_list
                else:
                    #add to the first zero pos and increment the indexer:
                    e[first_zero] = i
                    first_zero += 1
                graph[i] = graph[:,i] = 0
                deleted[i] = True
                modified[neighbours] = 1
                print("rule 5")
            elif (valency <= m): 
                bool_subset, j_node = check_subset(graph, neighbours) #gamma(i) \subset j^uptack, j \in gamma(i)
                if bool_subset:
                    merge_forest[j_node][i] = 1 #merge i into j, add directed edge j->i
                    w[j_node] += 1 #increment weight
                    graph[i] = graph[:,i] = 0
                    deleted[i] = True
                    modified[neighbours] = 1
                    print("rule 6")
                    print(neighbours, modified[neighbours])
                    print("merged",i,j_node)
            modified[i] = 0 #set vertex as unmodified
            print("w,deleted",w, np.where(deleted == True)[0])
            print()
    #return e, w, first_zero, last_zero, merge_forest
    #return first_zero, last_zero

In [38]:
'''merge-forest placement test'''
dummy_order = np.array([7,9,8,-1,-1,-1,-1,-1,5,6])
dummy_placed = 3
dum_fz = 3
dum_lz = -3

#topological sort:
t = np.array([[0,0,1,0,0], #0-2,1-3,4-2
              [0,0,0,1,0],
              [0,0,0,0,0],
              [0,0,0,0,0],
              [0,0,1,0,0]
             ])
'''
t = np.array([[0,1,0,0],
              [0,0,1,0],
              [0,0,0,1],
              [0,0,0,0]])
'''


ordered_list = get_ordered_list_merged_vertex(t, dummy_placed)
print(dummy_order,ordered_list)
#place first:
length = len(ordered_list)
length_do = len(dummy_order)
print(length_do, length)
dummy_order[dum_fz : dum_fz + length] = ordered_list #firstzero placement
print(length_do + dum_lz - length, length_do + dum_lz + 1) #5,8
dummy_order[length_do + dum_lz - length + 1: length_do + dum_lz + 1] = ordered_list #lastzero placement
print(dummy_order)

S [2, 3]
[ 7  9  8 -1 -1 -1 -1 -1  5  6] [3, 1]
10 2
5 8
[ 7  9  8 -1 -1 -1  3  1  5  6]


In [3]:
'''graphs collection for normalization testing'''
#test array/graph from https://people.sc.fsu.edu/~jburkardt/m_src/rcm/rcm.html
graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])
print(graph)

#nauru graph
graph_1 = np.zeros((24,24))
graph_1[0][1] = graph_1[0][5] = graph_1[0][21] = \
graph_1[1][3] = graph_1[1][15] = \
graph_1[2][3] = graph_1[2][4] = graph_1[2][23] = \
graph_1[3][9] = graph_1[4][5] = graph_1[4][17] = \
graph_1[5][11] = graph_1[6][7] = graph_1[6][11] = graph_1[6][19] = \
graph_1[7][9] = graph_1[7][13] = \
graph_1[8][9] = graph_1[8][10] = graph_1[8][22] = \
graph_1[10][11] = graph_1[10][16] = \
graph_1[12][13] = graph_1[12][17] = graph_1[12][18] = \
graph_1[13][15] = graph_1[14][15] = graph_1[14][16] = graph_1[14][20] = \
graph_1[16][17] = graph_1[18][19] = graph_1[18][23] = \
graph_1[19][21] = graph_1[20][21] = graph_1[20][22] = \
graph_1[22][23] = 1
graph_1 += graph_1.T
print(np.allclose(graph_1, graph_1.T, rtol=1e-05, atol=1e-08))

'''testing-ground for normalize stage'''
e, w, first_zero, last_zero, merge_forest, deleted = initialize(graph)
print(normalize(graph))
print("e, w, first_zero, last_zero, merge_forest, deleted",e, w, first_zero, last_zero, merge_forest, deleted)

e, w, first_zero, last_zero, merge_forest, deleted = initialize(graph_1)
print(normalize(graph_1))
print("e, w, first_zero, last_zero, merge_forest, deleted",e, w, first_zero, last_zero, merge_forest, deleted)


[[0 0 1 0 0 0 0 1 1]
 [0 0 1 0 0 1 1 1 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]]
True
i, n, m, valency, valencies, e, modified, firstzero, lastzero

0 9 2.6666666666666665 3 [3 4 2 2 2 2 3 4 2] [-1 -1 -1 -1 -1 -1 -1 -1 -1] [1 1 1 1 1 1 1 1 1] 9 0 -1
neighbours [2 7 8]
w,deleted [1 1 1 1 1 1 1 1 1] []

1 9 2.6666666666666665 4 [3 4 2 2 2 2 3 4 2] [-1 -1 -1 -1 -1 -1 -1 -1 -1] [0 1 1 1 1 1 1 1 1] 8 0 -1
neighbours [2 5 6 7]
w,deleted [1 1 1 1 1 1 1 1 1] []

2 9 2.6666666666666665 2 [3 4 2 2 2 2 3 4 2] [-1 -1 -1 -1 -1 -1 -1 -1 -1] [0 0 1 1 1 1 1 1 1] 7 0 -1
neighbours [0 1]
rule 4
w,deleted [1 1 1 1 1 1 1 1 1] [2]

3 8 2.75 2 [3 4 0 2 2 2 3 4 2] [ 2 -1 -1 -1 -1 -1 -1 -1 -1] [1 1 0 1 1 1 1 1 1] 8 1 -1
neighbours [7 8]
rule 4
w,deleted [1 1 1 1 1 1 1 1 1] [2 3]

4 7 2.857142857142857 2 [3 4 0 0 2 2 3 4 2] [ 2  3 -1 -1 -1 -1 -1 -1 -1] [1 1 0 0 1 1 1 1 1] 7 2 -1
neighbours [6 7]
rule 4


C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:195: RuntimeWarning: invalid value encountered in long_scalars


In [108]:
"""testing the merge rule (r6) using banded matrix"""
graph_2 = np.array([[0,1,1,1,0,0,0],
                    [1,0,1,1,1,0,0],
                    [1,1,0,1,1,1,0],
                    [1,1,1,0,1,1,1],
                    [0,1,1,1,0,1,1],
                    [0,0,1,1,1,0,1],
                    [0,0,0,1,1,1,0]
                   ])
print("g2:")
print("symm", np.allclose(graph_2, graph_2.T, rtol=1e-05, atol=1e-08))
print("clique", clique_check(graph_2, [1,2,3]))
print("1^up_tack", check_subset(graph_2, [1,2,3]))
e, w, first_zero, last_zero, merge_forest = initialize(graph_2)
print(normalize(graph_2, e, w, first_zero, last_zero, merge_forest))


graph_3 = np.array([[0,1,1,1],
                    [1,0,1,0],
                    [1,1,0,1],
                    [1,0,1,0]])
print("g3:")
print("clique", clique_check(graph_3, [1,2,3]))
print("1^up_tack", check_subset(graph_3, [1,2,3]))

#test using another bamded matrix
from scipy.sparse import diags
graph_4 = diags([1,1,1, 1, 0, 1, 1,1,1], [-4,-3,-2,-1, 0, 1,2,3,4], shape=(8, 8), dtype=int).toarray()
print("g4:")
print(graph_4)
print("clique", clique_check(graph_4, [1,2,3]))
print("1^up_tack", check_subset(graph_4, [1,2,3]))
e, w, first_zero, last_zero, merge_forest = initialize(graph_4)
print(normalize(graph_4, e, w, first_zero, last_zero, merge_forest))

g2:
symm True
clique True
1^up_tack (True, 1)
i, n, m, valency, valencies, e, modified, firstzero, lastzero

4.285714285714286 4.0
0 7 4.0 3 [3 4 5 6 5 4 3] [-1 -1 -1 -1 -1 -1 -1] [1 1 1 1 1 1 1] 7 0 -1
rule 5
w [1 1 1 1 1 1 1]

4.0 4.0
1 6 4.0 3 [0 3 4 5 5 4 3] [ 0 -1 -1 -1 -1 -1 -1] [0 1 1 1 1 1 1] 6 1 -1
rule 5
w [1 1 1 1 1 1 1]

3.6 4.0
2 5 3.6 3 [0 0 3 4 4 4 3] [ 0  1 -1 -1 -1 -1 -1] [0 0 1 1 1 1 1] 5 2 -1
rule 5
w [1 1 1 1 1 1 1]

3.0 4.0
3 4 3.0 3 [0 0 0 3 3 3 3] [ 0  1  2 -1 -1 -1 -1] [0 0 0 1 1 1 1] 4 3 -1
rule 1
w [1 1 1 1 1 1 1]

2.0 4.0
4 3 2.0 2 [0 0 0 0 2 2 2] [ 0  1  2 -1 -1 -1  3] [0 0 0 0 1 1 1] 3 3 -2
rule 1
w [1 1 1 1 1 1 1]

1.0 4.0
5 2 1.0 1 [0 0 0 0 0 1 1] [ 0  1  2 -1 -1  4  3] [0 0 0 0 0 1 1] 2 3 -3
rule 1
w [1 1 1 1 1 1 1]

nan 3.0
6 0 nan 0 [0 0 0 0 0 0 0] [ 0  1  2 -1  5  4  3] [0 0 0 0 0 0 1] 1 3 -4
rule 3
w [1 1 1 1 1 1 1]

[0 1 2 6 5 4 3]
g3:
clique False
1^up_tack (True, 2)
g4:
[[0 1 1 1 1 0 0 0]
 [1 0 1 1 1 1 0 0]
 [1 1 0 1 1 1 1 0]
 [1 1 1 0 1 1 1 1]
 [

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:189: RuntimeWarning: invalid value encountered in long_scalars


#### Separation Stage

In [57]:
'''dijkstra https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm''' 
#function to help djikstra algorithm:
def get_min_distance_vertex(Q, distances):
    min_dist = float("inf")
    min_v = None
    for v in range(Q.shape[0]):
        if (distances[v] < min_dist) and (Q[v] == 1):
            min_dist = distances[v]
            min_v = v
    return min_dist, min_v

#start of dijkstra algorithm:
def dijkstra_shortest_path(graph, source):
    #n_init = get_total_nodes(graph, graph.shape[0])
    n_init = graph.shape[0]
    Q = np.array([1]*n_init)
    #print(Q)
    #source = root = 0
    distances = np.array([float("inf")]*n_init) #set distance vector
    distances[source] = 0
    prev = np.array([None]*n_init)

    while np.sum(Q) > 0:
        _, u = get_min_distance_vertex(Q, distances) #get the vertex with minimum distance
        Q[u] = False #remove u from Q
        neighbours = np.where(graph[u] == 1)[0]
#        print("len(Q), neighbours",len(Q), neighbours)
        for v in neighbours:
            if Q[v] == 1:
                alt = distances[u] + graph[u][v] #distance is equal to the weight of the edge between u and v
                if alt < distances[v]:
                    distances[v] = alt
                    prev[v] = u
                #print(alt)
    return distances, prev

#function to find max valency from nodes
def get_max_valency(subset_nodes, valencies):
    max_valency = -float("inf")
    max_vertex = None
    for m in subset_nodes:
        if valencies[m] > max_valency:
            max_valency = valencies[m]
            max_vertex = m
    return max_vertex, max_valency

'''Separate Stage'''
def separate(graph):
    global deleted, e, w, first_zero, last_zero, merge_forest
    n_init = graph.shape[0] #actual graph size
    
    '''RCM part'''
    #1, d=0, pick vertex e with max valency:
    print("#1: ")
    d_prime = 0
    n_nodes = get_total_nodes(graph, graph.shape[0]) #current total nodes
    valencies = np.array([np.sum(graph[i]) for i in range(n_init)])
    e_sep = np.argmax(valencies) #get the node with max valency

    #2, need to find a set of M with max distansce from e, which requires BFS or djikstra:
    print("#2: ")
    distances, _ = dijkstra_shortest_path(graph, e_sep)
    print("distances",distances)
    conn_components = np.where(distances != np.inf)[0] #indexes of connected components within the subgraph where e resides
    conn_distances = distances[conn_components] #distances of connected components (distances excluding inf)
    s = conn_components.shape[0] #total connected components
    d = np.max(conn_distances) #max distance from e
    M = np.where(distances == d)[0] #set of vertices with max distance from e
    print("n_init, valencies, e_sep, s, d, M, conn_distances")
    print(n_init, valencies, e_sep, s, d, M, conn_distances)
    
    
    #3, if d'>d, d'=d, pick a vertex from M with max valency, back to 2 if the first e is close to the second e
    print("#3: ")
    loopcount = 0 #for repetition statistics
    while d>d_prime:
        print("d, d_prime, e_sep",d, d_prime, e_sep)
        d_prime = d
        max_vertex,_ = get_max_valency(M, valencies)
        print("M, valencies",M, valencies)
        e_sep = max_vertex

        #do 2 again:
        distances, _ = dijkstra_shortest_path(graph, e_sep)
        conn_components = np.where(distances != np.inf)[0] #indexes of connected components within the subgraph where e resides
        conn_distances = distances[conn_components] #distances of connected components (distances excluding inf)
        s = conn_components.shape[0] #total connected components
        d = np.max(conn_distances) #max distance from e
        M = np.where(distances == d)[0] #set of vertices with max distance from e
        loopcount+=1
    print("RCM loopcount", loopcount)
    print("n_init, valencies, e_sep, s, d, M, conn_distances")
    print(n_init, valencies, e_sep, s, d, M, conn_distances)
    '''end of RCM'''

    
    #3.5, get the n_k from e, 0<=k<=d, d=max distance, k \in Z
    print("#3.5: n_k from e, 0<=k<=d, d=max distance")
    max_d = np.max(conn_distances).astype(int)
    n_arr = np.zeros(max_d+1)
    for i in range(0,max_d+1):
        n_arr[i] = np.where(conn_distances == i)[0].shape[0]
    print("n_arr",n_arr)
    
    
    #4, initialization of several variables:
    ##NOTE: there are two n's, n_k and n_{k+1}, which will be used for comparison in a condition.
    print("#4: ")
    k=0; N=[np.array([e_sep])]; n=[1]; u = s-1; tried = np.array([0]*n_init); tried[e_sep] = 1
    
    seploop = 0
    while True:
        #first line:
        #gamma_{k+1}(e):=get neighbours/set of points from e with the distance of k+1
        N_next = np.where(distances == k+1)[0] #get the set of neighbours with distance = k+1
        N.append(N_next)
        n.append(len(N[k+1])) #or sum of weights?
        u -= n[k+1]
        print("k,N,n,u",k,N,n,u)

        #print("n_arr[k] <= n_arr[k+1] < n_arr[k+2]",n_arr[k] <= n_arr[k+1] < n_arr[k+2])
        
        print("k+2, len(n_arr), d",k+2, len(n_arr), d)
        '''if k+2 < len(n_arr): #temporary fix, by skipping the block if k+2 >= len(n)
            if (n_arr[k] <= n_arr[k+1] < n_arr[k+2]):
                k += 1
                continue'''
        if (k < d-1) and (n_arr[k] <= n_arr[k+1] < n_arr[k+2]) and (u > 0.4*s): #another fix, by adding more skip-conditions
            print("(k < d-1) and (n_arr[k] <= n_arr[k+1] < n_arr[k+2]) and (u > 0.4*s) condition reached")
            k += 1
            continue
        
        #second line, determining "in degrees":
        #c = {} #need to know which c corresponds to which node, so probably use a dictionary
        c = np.zeros(n_init)
        j_idxs = N[k+1] #to keep track the used node indexes
        for node_j in N[k+1]: #indexing not by nodes' indices, but by c's internal index
            gamma_j = np.where(graph[node_j] == 1)[0]
            c[node_j] = (np.intersect1d(gamma_j, N[k])).shape[0]
            #print("gamma_j, N[k]",gamma_j, N[k])
        print("c[j_idxs]",c[j_idxs])

        #third line, determining the "out-weights" (weights from normalization stage):
        #b = {} #same reason with c
        b = np.zeros(n_init)
        i_idxs = N[k]
        for node_i in N[k]:
            gamma_i = np.where(graph[node_i] == 1)[0]
            out_w_nodes = np.intersect1d(gamma_i, N[k+1])
            b[node_i] = np.sum(w[out_w_nodes]) #w = weights from normalization, need to know which value belongs to which
            print("gamma_i, N[k+1]",gamma_i, N[k+1])
        print("b",b)
        #new condition to check, when b_i = 0, then break:
        if np.sum(b) == 0:
            print("b_i all zero")
            print("k,d,b,c",k,d,b,c)
            break
        #
        #fourth line:
        while n[k] > 0:
            print("n[k]>0",n[k] > 0)
            if (u > 0.4*s) and (n[k+1] < n[k]): #threshold = 0.4s
                print("(u > 0.4*s) and (n[k+1] < n[k])",(u > 0.4*s) and (n[k+1] < n[k]))
                break
            #place i with largest b_i last: (the rule should follow the placement rule in normalization)
            placed = np.argmax(b)
            print("placed",placed)
            ##start of temporary fix
            #if b[placed] > 0: #meaning, gamma(i) \intersect N_{k+1} is not {}
            if w[placed] > 1:
                ordered_list = get_ordered_list_merged_vertex(merge_forest, placed)
                len_e = len(e)
                len_ord_list = len(ordered_list)
                e[len_e + last_zero - len_ord_list + 1 : len_e + last_zero + 1] = ordered_list
                last_zero -= len_ord_list
            else:
                e[last_zero] = placed
                last_zero -= 1
            graph[placed] = graph[:,placed] = 0
            deleted[placed] = True
            b[placed] = 0 #remove from b
            #decrement s, n_k, c_j:
            #print("s,n[k],c",s,n[k],c)
            s -= 1; n[k] -= 1; c[N[k+1]] -= 1
            ##end of temporary fix#
            #print("s,n[k],c",s,n[k],c)
            #if c_j == 0: ......
            #drop c_j from N; incr u; decr n[k+1]:
            for node_j in N[k+1]:
                if c[node_j] == 0:
                    N[k+1] = N[k+1][N[k+1] != node_j] #drop cj from N
                    u += 1; n[k+1] -= 1
                    #print("N, u, n, c[node_j]",N, u, n, c[node_j])
        if n[k] == 0: ##NOTE: this part is a little bit uncanny, since in first iter the n[k] will always reach 0
            break
        tried[N[k]] = 1; k += 1 #mark all i \in N_k as tried, increment k
        seploop+=1
        print("\n seploop",seploop)

    print(e)
    #return graph, e, w, first_zero, last_zero, merge_forest
    return first_zero, last_zero
            #break #for loop breaking purpose during tests -- removed on actual scenario
        #break #for loop breaking purpose during tests -- removed on actual scenario

'''dummy data'''
graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])

#nauru graph (bipartite)
graph_1 = np.zeros((24,24))
graph_1[0][1] = graph_1[0][5] = graph_1[0][21] = \
graph_1[1][3] = graph_1[1][15] = \
graph_1[2][3] = graph_1[2][4] = graph_1[2][23] = \
graph_1[3][9] = graph_1[4][5] = graph_1[4][17] = \
graph_1[5][11] = graph_1[6][7] = graph_1[6][11] = graph_1[6][19] = \
graph_1[7][9] = graph_1[7][13] = \
graph_1[8][9] = graph_1[8][10] = graph_1[8][22] = \
graph_1[10][11] = graph_1[10][16] = \
graph_1[12][13] = graph_1[12][17] = graph_1[12][18] = \
graph_1[13][15] = graph_1[14][15] = graph_1[14][16] = graph_1[14][20] = \
graph_1[16][17] = graph_1[18][19] = graph_1[18][23] = \
graph_1[19][21] = graph_1[20][21] = graph_1[20][22] = \
graph_1[22][23] = 1
graph_1 += graph_1.T
#print(np.allclose(graph_1, graph_1.T, rtol=1e-05, atol=1e-08))
graph = graph_1

'''
graph = np.array([[0,1,1,1,0,0,0],
                    [1,0,1,1,1,0,0],
                    [1,1,0,1,1,1,0],
                    [1,1,1,0,1,1,1],
                    [0,1,1,1,0,1,1],
                    [0,0,1,1,1,0,1],
                    [0,0,0,1,1,1,0]
                   ])
'''

'''n_init = graph.shape[0]
w = np.array([1]*n_init)
first_zero = 0; last_zero = -1
e = np.zeros(n_init)'''
e,w,first_zero, last_zero, merge_forest, deleted = initialize(graph) 
'''end of dummy data'''

separate(graph)
print("e,w,first_zero, last_zero, merge_forest, deleted: \n",e,w,first_zero, last_zero, merge_forest, np.where(deleted == True))

#1: 
#2: 
distances [0. 1. 3. 2. 2. 1. 3. 4. 4. 3. 3. 2. 4. 3. 3. 2. 4. 3. 3. 2. 2. 1. 3. 4.]
n_init, valencies, e_sep, s, d, M, conn_distances
24 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.] 0 24 4.0 [ 7  8 12 16 23] [0. 1. 3. 2. 2. 1. 3. 4. 4. 3. 3. 2. 4. 3. 3. 2. 4. 3. 3. 2. 2. 1. 3. 4.]
#3: 
d, d_prime, e_sep 4.0 0 0
M, valencies [ 7  8 12 16 23] [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
RCM loopcount 1
n_init, valencies, e_sep, s, d, M, conn_distances
24 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.] 7 24 4.0 [ 0  4 16 20 23] [4. 3. 3. 2. 4. 3. 1. 0. 2. 1. 3. 2. 2. 1. 3. 2. 4. 3. 3. 2. 4. 3. 3. 4.]
#3.5: n_k from e, 0<=k<=d, d=max distance
n_arr [1. 3. 6. 9. 5.]
#4: 
k,N,n,u 0 [array([7], dtype=int64), array([ 6,  9, 13], dtype=int64)] [1, 3] 20
k+2, len(n_arr), d 2 5 4.0
(k < d-1) and (n_arr[k] <= n_arr[k+1] < n_arr[k+2]) and (u > 0.4*s) condition reached
k,N,n,u 1 [array([7], dtype=int64), ar

### Combining both normalize and separate stage

In [53]:
'''input data'''
'''graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])'''
#nauru graph (bipartite)
graph_1 = np.zeros((24,24))
graph_1[0][1] = graph_1[0][5] = graph_1[0][21] = \
graph_1[1][3] = graph_1[1][15] = \
graph_1[2][3] = graph_1[2][4] = graph_1[2][23] = \
graph_1[3][9] = graph_1[4][5] = graph_1[4][17] = \
graph_1[5][11] = graph_1[6][7] = graph_1[6][11] = graph_1[6][19] = \
graph_1[7][9] = graph_1[7][13] = \
graph_1[8][9] = graph_1[8][10] = graph_1[8][22] = \
graph_1[10][11] = graph_1[10][16] = \
graph_1[12][13] = graph_1[12][17] = graph_1[12][18] = \
graph_1[13][15] = graph_1[14][15] = graph_1[14][16] = graph_1[14][20] = \
graph_1[16][17] = graph_1[18][19] = graph_1[18][23] = \
graph_1[19][21] = graph_1[20][21] = graph_1[20][22] = \
graph_1[22][23] = 1
graph_1 += graph_1.T
#print(np.allclose(graph_1, graph_1.T, rtol=1e-05, atol=1e-08))

#custom graph
graph_2 = np.array([
    [0,1,0,0,1,0,1,1],
    [1,0,0,0,1,0,0,1],
    [0,0,0,0,0,0,0,1],
    [0,0,0,0,1,1,1,1],
    [1,1,0,1,0,1,1,1],
    [0,0,0,1,1,0,1,0],
    [1,0,0,1,1,1,0,1],
    [1,1,1,1,1,0,1,0]
])


def elimination_ordering(graph):
    #alternate normalize and separate while the graph is not empty
    while np.sum(graph) > 0:
        if np.sum(graph) == 0:
            break
        print("Normalize:")
        normalize(graph)
        print("e, w, first_zero, last_zero, deleted", e, w, first_zero, last_zero, np.where(deleted == True))
        if np.sum(graph) == 0:
            break
        print("\n Separate:")
        separate(graph)
        print("e, w, first_zero, last_zero, deleted \n", e, w, first_zero, last_zero, np.where(deleted == True))
        #print(graph, merge_forest)
        print("==================NEW ROUND======================= \n")
    return e

'''initialization'''
graph = graph_2
graph_elim = np.copy(graph)
graph_elim2 = np.copy(graph_elim)
e, w, first_zero, last_zero, merge_forest, deleted = initialize(graph) #must be on global scope

'''ordered elimination'''
elimination_ordering(graph)
fills, _ = eliminate(graph_elim, e)
print("eli",fills,e)

'''test using metis'''
#adj_mat_to_metis_file(graph_elim2, "custgraph.metisgraph")
#metis_order = iperm_to_orderlist("matrices/nauru_bipartite.metisgraph.iperm")
metis_order = iperm_to_orderlist("matrices/custgraph.metisgraph.iperm")
fills, _ = eliminate(graph_elim2, metis_order)
print("metis", fills, metis_order)

Normalize:
i, n, m, valency, valencies, e, modified, firstzero, lastzero

mean_valency, np.floor(n**(1/4) + 3) 4.0 4.0
0 8 4.0 4 [4 3 1 4 6 3 5 6] [-1 -1 -1 -1 -1 -1 -1 -1] [1 1 1 1 1 1 1 1] 8 0 -1
neighbours [1 4 6 7]
rule 6
[1 4 6 7] [1 1 1 1]
merged 0 4
w,deleted [1 1 1 1 2 1 1 1] [0]

mean_valency, np.floor(n**(1/4) + 3) 3.4285714285714284 4.0
1 7 3.4285714285714284 2 [0 2 1 4 5 3 4 5] [-1 -1 -1 -1 -1 -1 -1 -1] [0 1 1 1 1 1 1 1] 7 0 -1
neighbours [4 7]
rule 4
w,deleted [1 1 1 1 2 1 1 1] [0 1]

mean_valency, np.floor(n**(1/4) + 3) 3.3333333333333335 4.0
2 6 3.3333333333333335 1 [0 0 1 4 4 3 4 4] [ 1 -1 -1 -1 -1 -1 -1 -1] [0 0 1 1 1 1 1 1] 6 1 -1
neighbours [7]
rule 3
w,deleted [1 1 1 1 2 1 1 1] [0 1 2]

mean_valency, np.floor(n**(1/4) + 3) 3.6 4.0
3 5 3.6 4 [0 0 0 4 4 3 4 3] [ 1  2 -1 -1 -1 -1 -1 -1] [0 0 0 1 1 1 1 1] 5 2 -1
neighbours [4 5 6 7]
rule 1
w,deleted [1 1 1 1 2 1 1 1] [0 1 2 3]

mean_valency, np.floor(n**(1/4) + 3) 2.5 4.0
4 4 2.5 3 [0 0 0 0 3 2 3 2] [ 1  2 -1 -1 -1 -1 -

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:195: RuntimeWarning: invalid value encountered in long_scalars


### Utilities

In [42]:
def eliminate(graph, elimination_order):
    '''elimination function: eliminates the vertices based on the order resulted from elimination ordering algorithms
    - takes in the vertices order from the any elimination ordering algorithms (e.g. METIS' nested dissection)
    - fill will be added when the "center" of the vertex is eliminated, e.g., 1-2-3, eliminate 2, fill(1-3), fill_count+=1
    - for now, assume the fill will be comb(n,2), so if there are 3 vertices which depend on an eliminated vertex, there will be 6 fills
    '''
    count_fill = 0
    for node_i in elimination_order:
        #find neighbours and fill the fill-in indexes:
        neighbours = np.where(graph[node_i] == 1)[0]
        fill_idxs = list(combinations(neighbours, 2))
        #fill-in the edge of i's neighbours:
        if len(fill_idxs) > 0:
            #check if the edges are present, if there are no edges, add them:
            for fill in fill_idxs:
                if graph[fill[0]][fill[1]] == 0:
                    graph[fill[0]][fill[1]] = graph[fill[1]][fill[0]] = 1
                    count_fill += 1
        #remove the edges of i:
        graph[node_i] = graph[:,node_i] = 0
    return count_fill, graph

def adj_mat_to_metis_file(graph, filename):
    '''write adjacency matrix to file'''
    first_line = np.array([graph.shape[0], int(np.sum(graph)/2)]) #[nodes, edges]
    adj_list = []
    for i in range(graph.shape[0]):
        neighbours = np.where(graph[i] == 1)[0]
        neighbours += 1
        adj_list.append(neighbours)
    adj_list = np.array(adj_list)
    
    with open(filename,"w") as f:
        f.write(str(first_line[0])+" "+str(first_line[1])+"\n")
        for i in range(adj_list.shape[0]):
            for j in range(adj_list[i].shape[0]):
                f.write(str(adj_list[i][j])+" ")
            f.write("\n")
    print("writing",filename,"done!")

def iperm_to_orderlist(filename):
    '''read iperm from ndmetis and convert it to list'''
    f = open(filename, "r")
    order = []
    for x in f:
        order.append(int(x))
    order = np.array(order)
    #according to metis documentatoin:
    actual_order = np.zeros(order.shape[0])
    for i in range(order.shape[0]):
        actual_order[i] = np.where(order == i)[0]
    actual_order = actual_order.astype(np.int64, copy=False)
    return actual_order

def load_matrix_market(filename):
    '''test using matrices from matrix market'''
    #filename = "matrices/bcsstm01.mtx.gz"
    print(mminfo(filename))
    Matrix = mmread(filename)
    A = Matrix.toarray()
    #print(A)
    #print(np.nonzero(A))
    '''preprocess the matrix'''
    A = A.astype(np.int64, copy=False)
    #symmetrize the matrix:
    A = A + A.T
    #print("symmetrize:")
    #print(A)
    #set diagonals to zero:
    np.fill_diagonal(A, 0)
    #print("diag")
    #print(A)
    #if a nonzero element is >0 or <0, set it to 1:
    #print("nz")
    A[np.nonzero(A)] = 1
    #print(A)
    return A

graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])
order = [2, 3, 4, 5, 6, 8, 1, 0, 7] #from the elimination ordering result
countfill,_ = eliminate(graph, order)
print("total fill:" ,countfill)

#banded matrix 2m+1 = 15
graph_2 = np.array([[0,1,1,1,0,0,0],
                    [1,0,1,1,1,0,0],
                    [1,1,0,1,1,1,0],
                    [1,1,1,0,1,1,1],
                    [0,1,1,1,0,1,1],
                    [0,0,1,1,1,0,1],
                    [0,0,0,1,1,1,0]
                   ])
order = [0, 1, 2, 6, 5, 4, 3]
countfill,_ = eliminate(graph, order)
print("total fill:" ,countfill)

print(iperm_to_orderlist("matrices/disconn_graph.metisgraph.iperm"))

A = load_matrix_market("matrices/bcsstk01.mtx.gz")
print(np.nonzero(A))

total fill: 3
total fill: 0
[7 1 3 0 2 6 5 4]


FileNotFoundError: [Errno 2] No such file or directory: 'matrices/bcsstk01.mtx.gz'

### Testing schemes

In [17]:
'''matrices pre-processing'''
mypath = "matrices/mm/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
#for all matrices, load:
for f in onlyfiles:
    G = load_matrix_market("matrices/mm/"+f)
    #print(np.diagonal(G))
    print(np.allclose(G, G.T, rtol=1e-05, atol=1e-08))

    #save all matrices to metis format:
    adj_mat_to_metis_file(G, "matrices/ndmetis_input/"+f+".metisgraph")

    #run the metis using all the matrices elsewhere.....



['bcsstk01.mtx.gz', 'bcsstm13.mtx.gz', 'mcca.mtx.gz', 'mcfe.mtx.gz']
(48, 48, 224, 'coordinate', 'real', 'symmetric')
True
writing matrices/ndmetis_input/bcsstk01.mtx.gz.metisgraph done!
(2003, 2003, 11973, 'coordinate', 'real', 'symmetric')
True
writing matrices/ndmetis_input/bcsstm13.mtx.gz.metisgraph done!
(180, 180, 2659, 'coordinate', 'real', 'general')
True
writing matrices/ndmetis_input/mcca.mtx.gz.metisgraph done!
(765, 765, 24382, 'coordinate', 'real', 'general')
True
writing matrices/ndmetis_input/mcfe.mtx.gz.metisgraph done!


In [63]:
'''testing main for matrix market'''
#for all matrices, reload:
mypath = "matrices/mm/"
onlyfiles = np.array([f for f in listdir(mypath) if isfile(join(mypath, f))])
onlyfiles = onlyfiles[[2]]
print(onlyfiles)
fills_eli = []
fills_metis = []
for f in onlyfiles:
    G = load_matrix_market(mypath+f)
    
    #initialize: MUST always be in global scope:
    G1 = np.copy(G)
    G2 = np.copy(G)
    e, w, first_zero, last_zero, merge_forest, deleted = initialize(G)

    #elimination ordering:
    elimination_ordering(G)
    fills, _ = eliminate(G1, e)
    fills_eli.append(fills)
    print("eli",fills,e)
    print("sorted e", sorted(e))

    #metis:
    metis_order = iperm_to_orderlist("matrices/ndmetis_iperm/"+f+".metisgraph.iperm")
    fills, _ = eliminate(G2, metis_order)
    fills_metis.append(fills)
    print("metis", fills, metis_order)
    print("sorted e", sorted(metis_order))

print("mm",onlyfiles)
print("eli",fills_eli)
print("metis",fills_metis)

#check ordering result:
e = np.array(e)
e_sort = np.sort(e[e != -1])
print(e_sort)


['mcca.mtx.gz']
(180, 180, 2659, 'coordinate', 'real', 'general')
Normalize:
i, n, m, valency, valencies, e, modified, firstzero, lastzero

mean_valency, np.floor(n**(1/4) + 3) 18.666666666666668 6.0
0 180 6.0 3 [ 3  3  3  3  3  3  3  3  3  3  3  3  4  3  4  3  5  3 11  3  5  3 12  3
  5  3 15  3  8  3 16  3  9  3 23  3 10  5 30  3 15  7 36  3 16 15 43  3
 17 17 43  3 18 25 49  3 18 33 53  3 18 39 57  3 17 43 63  3 17 43 62  3
 16 41 62  3 14 41 61  3 14 43 61  3 14 43 60  3 19 43 62  3 19 41 60  3
 19 41 58  3 20 39 53  3 20 39 51  3 19 45 52  3  9 45 52  3  9 41 48  3
  9 43 50  3 10 41 49  3 10 43 49  3 10 41 47  3 10 27 33  3 10 21 29  3
 10 19 28  3  9 19 25  3  8 17 24  3  8 15 21  3  9 15 19  3  9 13 16  3
  8  9 13  3  4  7  9  3  4  5  5  3] [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -

 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 23 24 25
 27 29 31 33 35 37 39 43 47 51]

mean_valency, np.floor(n**(1/4) + 3) 21.972602739726028 6.0
53 146 6.0 25 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0  0  8  0
  0  0 10  0  6  0 12  0  7  0 20  0  8  0 28  0 14  6 35  0 15 13 41  0
 16 16 42  0 18 25 49  3 18 33 53  3 18 39 57  3 17 43 63  3 17 43 62  3
 16 41 62  3 14 41 61  3 14 43 61  3 14 43 60  3 19 43 62  3 19 41 60  3
 19 41 58  3 20 39 53  3 20 39 51  3 19 45 52  3  9 45 52  3  9 41 48  3
  9 43 50  3 10 41 49  3 10 43 49  3 10 41 47  3 10 27 33  3 10 21 29  3
 10 19 28  3  9 19 25  3  8 17 24  3  8 15 21  3  9 15 19  3  9 13 16  3
  8  9 13  3  4  7  9  3  4  5  5  3] [ 0  1  2  3  4  5  6  7  8  9 10 11 13 12 14 15 17 19 21 23 25 27 29 31
 33 35 39 43 47 51 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -

mean_valency, np.floor(n**(1/4) + 3) 23.147058823529413 6.0
95 136 6.0 3 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0  0  0  8  0
  0  0 10  0  6  0 12  0  7  0 20  0  8  0 28  0 14  6 35  0 15 13 41  0
 16 16 42  0 17 24 48  0 17 32 52  0 17 38 56  0 16 42 62  0 16 42 61  0
 15 40 61  0 13 40 60  0 13 42 60  0 13 42 59  0 18 42 61  0 19 41 60  3
 19 41 58  3 20 39 53  3 20 39 51  3 19 45 52  3  9 45 52  3  9 41 48  3
  9 43 50  3 10 41 49  3 10 43 49  3 10 41 47  3 10 27 33  3 10 21 29  3
 10 19 28  3  9 19 25  3  8 17 24  3  8 15 21  3  9 15 19  3  9 13 16  3
  8  9 13  3  4  7  9  3  4  5  5  3] [ 0  1  2  3  4  5  6  7  8  9 10 11 13 12 14 15 17 19 21 23 25 27 29 31
 33 35 39 43 47 51 55 59 63 67 71 75 79 83 87 91 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 

  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1
 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1
 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] 123 51 -1
neighbours [105 106 109 110 113 114 117 118 121 122 125 126 128 129 130 132 133 134
 136 137 139 140 141 142 144 145 146 148 149 150 152 153 154]
w,deleted [1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1
 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [  0   1   2   3   4   5   6

 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0] 107 64 -1
neighbours [22 26 32 34 36 38 40 42 44 46]
w,deleted [1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 3 1 1 1 2 1 1 1 1 1 1 1 1 1 1
 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 2 2 1] [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  23  24  25  27  28  29  31  33  35  37  39  43  47  51
  55  59  63  67  71  75  79  83  87  91  95  99 103 107 111 115 119 123
 127 131 135 139 143 147 151 155 159 163 167 171 172 173 175 176 177 178
 179]

already deleted: 31
mean_valency, np.floor(n**(1/4) + 3) 27.7196261682243 6.0
32 107 6.0 6 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  0
  0  0  8  0  0  0 10  0  6  0 18  0  8  0

...

mean_valency, np.floor(n**(1/4) + 3) 8.421052631578947 5.0
145 19 5.0 8 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  7  0  0  0  0  0  0  0  7  0  0  0  7  0  0  0  7  7  0
  0  8  8  0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  0
  0  8  9  0  0 10 10  0  0 10  0  0  0 10 10  0  0  9  0  0  0  7  8  0
  0  0  0  0  0  0  0  0  0  0  0  0] [  0   1   2   3   4   5   6   7   8   9  10  11  13  12  14  15  17  19
  21  23  25  27  29  31  33  35  39  43  47  51  55  59  63  67  71  75
  79  83  87  91  95  99 103 107 111 115 119 123 127 131 135 139 143 147
 151 155 159 163 167 171 175 178 179  28 174  22 169  40  60  76 108 112
  34 116 120 124  49  88 104 128  92  96 100 144 148 152  42 156 160  -1
  -1  -1  -1  

mean_valency, np.floor(n**(1/4) + 3) 7.411764705882353 5.0
121 17 5.0 6 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0
 0 0 6 0 0 0 6 6 0 0 6 6 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 8 0
 0 9 9 0 0 9 0 0 0 9 9 0 0 9 0 0 0 7 8 0 0 0 0 0 0 0 0 0 0 0 0 0] [  0   1   2   3   4   5   6   7   8   9  10  11  13  12  14  15  17  19
  21  23  25  27  29  31  33  35  39  43  47  51  55  59  63  67  71  75
  79  83  87  91  95  99 103 107 111 115 119 123 127 131 135 139 143 147
 151 155 159 163 167 171 175 178 179  28 174  22 169  40  60  76 108 112
  34 116 120 124  49  88 104 128  92  96 100 144 148 152  42 156 160  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1  -1  -1 141 101  97  93  8

mean_valency, np.floor(n**(1/4) + 3) 6.133333333333334 4.0
157 15 4.0 7 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0
 0 0 6 0 0 0 6 6 0 0 6 6 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 6 0
 0 7 0 0 0 0 0 0 0 7 7 0 0 7 0 0 0 5 6 0 0 0 0 0 0 0 0 0 0 0 0 0] [  0   1   2   3   4   5   6   7   8   9  10  11  13  12  14  15  17  19
  21  23  25  27  29  31  33  35  39  43  47  51  55  59  63  67  71  75
  79  83  87  91  95  99 103 107 111 115 119 123 127 131 135 139 143 147
 151 155 159 163 167 171 175 178 179  28 174  22 169  40  60  76 108 112
  34 116 120 124  49  88 104 128  92  96 100 144 148 152  42 156 160  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1 153 150 141 101  97  93  8

eli 2009 [  0   1   2   3   4   5   6   7   8   9  10  11  13  12  14  15  17  19
  21  23  25  27  29  31  33  35  39  43  47  51  55  59  63  67  71  75
  79  83  87  91  95  99 103 107 111 115 119 123 127 131 135 139 143 147
 151 155 159 163 167 171 175 178 179  28 174  22 169  40  60  76 108 112
  34 116 120 124  49  88 104 128  92  96 100 144 148 152  42 156 160 145
 146 149 158 161 165 166 125  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1 122
 121 118 117 109 113 157 153 150 141 101  97  93  85 137 162 105  81  73
 154 129 133 142 138  46  77  69  65  89 114 110  53  57 134 130 126   0
   0 132  61 102 106  50  54  58  62  98  94  86  82  78  90  74  70  66]
sorted e [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 21, 22, 23, 25, 27, 28, 29, 31, 33, 34, 35, 39, 40, 42, 43, 46, 47, 49

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:195: RuntimeWarning: invalid value encountered in longlong_scalars


In [66]:
'''above cell must be run first'''
#e_sort = e_sort[2:]
print(e_sort)
#check missing:
def printMissingElements(arr, N): 
    # Initialize diff 
    diff = arr[0] 
    for i in range(N): 
        # Check if diff and arr[i]-i 
        # both are equal or not 
        if(arr[i] - i != diff): 
            # Loop for consecutive 
            # missing elements 
            while(diff < arr[i] - i): 
                print(i + diff, end = " ") 
                diff += 1
printMissingElements(e_sort, e_sort.shape[0])

[  2   3   4   5   6   7   8   9  10  11  12  13  14  15  17  19  21  22
  23  25  27  28  29  31  33  34  35  39  40  42  43  46  47  49  50  51
  53  54  55  57  58  59  60  61  62  63  65  66  67  69  70  71  73  74
  75  76  77  78  79  81  82  83  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130
 131 132 133 134 135 137 138 139 141 142 143 144 145 146 147 148 149 150
 151 152 153 154 155 156 157 158 159 160 161 162 163 165 166 167 169 171
 174 175 178 179]
16 18 20 24 26 30 32 36 37 38 41 44 45 48 52 56 64 68 72 80 84 136 140 164 168 170 172 173 176 177 

In [91]:
'''disconnected graph checking, if it is assumed as different \gamma'''
G = np.array([
        [0,0,0,1,1,0,0,0],
        [0,0,1,0,0,0,0,0],
        [0,1,0,0,0,0,0,0],
        [1,0,0,0,1,0,0,0],
        [1,0,0,1,0,0,0,0],
        [0,0,0,0,0,0,1,1],
        [0,0,0,0,0,1,0,1],
        [0,0,0,0,0,1,1,0]
    ])

G = np.array([
        [0,1,0,1,0,0,0,1],
        [1,0,0,1,1,0,0,0],
        [0,0,0,0,0,1,1,0],
        [1,1,0,0,1,0,0,1],
        [0,1,0,1,0,0,0,1],
        [0,0,1,0,0,0,1,0],
        [0,0,1,0,0,1,0,0],
        [1,0,0,1,1,0,0,0]
        
    ])
G_elim = np.copy(G)
visited = np.array([False]*G.shape[0])
traverse_count = 0
sub_Gs = []
for i in range(G.shape[0]):
    if not visited[i]:
        sub_G = BFS(G,i)
        sub_Gs.append(sub_G)
        visited[sub_G] = True
print(sub_Gs)

'''generating subgraphs from a whole matrix graph'''
'''Gs = []
for i in range(len(sub_Gs)):
    #set all indices except i to 0
    sliced = []
    for j in range(len(sub_Gs)):
        if j!=i:
            sliced.append(sub_Gs[j])
    print(sliced)
    sliced = [item for sublist in sliced for item in sublist] #flattened list
    print(sliced)
    sub_graph = np.copy(G)
    sub_graph[sliced] = sub_graph[:,sliced] = 0
    Gs.append(sub_graph)
print(Gs)
for G in Gs:
    G_elim = np.copy(G)
    e,w,first_zero,last_zero,merge_forest,deleted = initialize(G)
    #normalize(G)
    separate(G)
    print("e",e)
    print(eliminate(G_elim,e))
    print()
'''
e,w,first_zero,last_zero,merge_forest,deleted = initialize(G_elim) #always initialize before normalize
normalize(G)
print(e)
elim_order_metis = iperm_to_orderlist("disconn_graph.metisgraph.iperm")
print("from ndmetis",elim_order_metis)
G_elim2 = np.copy(G_elim)
fills, _ = eliminate(G_elim, elim_order_metis) #using metis
print("fills",fills)
fills, _ = eliminate(G_elim2, e) #using eli
print("from eli normalize", e)
print("fills",fills)

[[0, 1, 3, 7, 4], [2, 5, 6]]
i, n, m, valency, valencies, e, modified, firstzero, lastzero

0 8 2.75 3 [3 3 2 4 3 2 2 3] [-1 -1 -1 -1 -1 -1 -1 -1] [1 1 1 1 1 1 1 1] 8 0 -1
neighbours [1 3 7]
w,deleted [1 1 1 1 1 1 1 1] []

1 8 2.75 3 [3 3 2 4 3 2 2 3] [-1 -1 -1 -1 -1 -1 -1 -1] [0 1 1 1 1 1 1 1] 7 0 -1
neighbours [0 3 4]
w,deleted [1 1 1 1 1 1 1 1] []

2 8 2.75 2 [3 3 2 4 3 2 2 3] [-1 -1 -1 -1 -1 -1 -1 -1] [0 0 1 1 1 1 1 1] 6 0 -1
neighbours [5 6]
rule 4
w,deleted [1 1 1 1 1 1 1 1] [2]

3 7 2.5714285714285716 4 [3 3 0 4 3 1 1 3] [ 2 -1 -1 -1 -1 -1 -1 -1] [0 0 0 1 1 1 1 1] 5 1 -1
neighbours [0 1 4 7]
w,deleted [1 1 1 1 1 1 1 1] [2]

4 7 2.5714285714285716 3 [3 3 0 4 3 1 1 3] [ 2 -1 -1 -1 -1 -1 -1 -1] [0 0 0 0 1 1 1 1] 4 1 -1
neighbours [1 3 7]
w,deleted [1 1 1 1 1 1 1 1] [2]

5 7 2.5714285714285716 1 [3 3 0 4 3 1 1 3] [ 2 -1 -1 -1 -1 -1 -1 -1] [0 0 0 0 0 1 1 1] 3 1 -1
neighbours [6]
rule 3
w,deleted [1 1 1 1 1 1 1 1] [2 5]

6 5 3.2 0 [3 3 0 4 3 0 0 3] [ 2  5 -1 -1 -1 -1 -1 -1] [0 0 0 0 0

C:\Users\beryl\Anaconda3\lib\site-packages\ipykernel_launcher.py:195: RuntimeWarning: invalid value encountered in long_scalars
